<a href="https://colab.research.google.com/github/DhruvSharma036/Sentiment-and-summary-tool/blob/main/summary_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import files
uploaded = files.upload()

Saving Untitled form.csv to Untitled form (1).csv


In [12]:
pip install transformers

In [13]:
pip install torch torchvision torchaudio

In [14]:
!pip install requests beautifulsoup4 pandas numpy

In [15]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
import requests
import torch
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [19]:
# Load the CSV file into a pandas DataFrame
import io
df = pd.read_csv(io.StringIO(uploaded['Untitled form (1).csv'].decode('utf-8')))

# Display the first few rows of the DataFrame to inspect the data
print(df.head())

# Assuming the CSV has a column named 'feedback' for sentiment analysis and 'article' for summarization
feedback_column = 'feedback'
article_column = 'article'

                        Timestamp  \
0  2024/07/11 6:47:37 pm GMT+5:30   
1  2024/07/11 6:57:50 pm GMT+5:30   

  How was your experience at the event? what all did you like/dislike about it?  \
0  The event was enjoyable overall. There were se...                              
1  he event was a delightful experience overall. ...                              

  Please give your suggestions  as to improve the event organisation and management  
0  Better communication, efficient registration, ...                                 
1  Suggestions for Improvement:\n\nStreamline the...                                 


In [23]:
# Load the CSV file into a pandas DataFrame, assuming a single column of data
import io
df = pd.read_csv(io.StringIO(uploaded['Untitled form (1).csv'].decode('utf-8')), header=None, names=['text'])  # Read without header, assign 'text' as column name

# Display the first few rows of the DataFrame to inspect the data
print(df.head())

# Use the 'text' column for both sentiment analysis and summarization
text_column = 'text'

# Function to get sentiment score
def get_sentiment_score(text):
    tokens = tokenizer.encode(text, return_tensors='pt')
    result = model(tokens)
    sentiment = int(torch.argmax(result.logits)) + 1  # Convert logits to sentiment score
    return sentiment

# Function to get summary
def get_summary(text):
    summary = summarizer(text, max_length=150, min_length=30, do_sample=False)
    return summary[0]['summary_text']

# Apply sentiment analysis and summarization to each row
df['sentiment_score'] = df[text_column].apply(get_sentiment_score)
df['summary'] = df[text_column].apply(get_summary)

# Display the resulting DataFrame
print(df[['text', 'sentiment_score', 'summary']])

print("\nSummaries:\n")
# Reset index to ensure 'idx' is an integer
for idx, row in df.reset_index().iterrows():
    print(f"Text {idx + 1}: {row['text']}")
    print(f"Summary: {row['summary']}\n")
    print("Sentiment Score:", row['sentiment_score'])

                                                                                                                                text
Timestamp                      How was your experience at the event? what all ...  Please give your suggestions  as to improve th...
2024/07/11 6:47:37 pm GMT+5:30 The event was enjoyable overall. There were sev...  Better communication, efficient registration, ...
2024/07/11 6:57:50 pm GMT+5:30 he event was a delightful experience overall. I...  Suggestions for Improvement:\n\nStreamline the...


Your max_length is set to 150, but your input_length is only 15. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Your max_length is set to 150, but your input_length is only 28. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)
Your max_length is set to 150, but your input_length is only 96. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=48)


                                                                                                                                text  \
Timestamp                      How was your experience at the event? what all ...  Please give your suggestions  as to improve th...   
2024/07/11 6:47:37 pm GMT+5:30 The event was enjoyable overall. There were sev...  Better communication, efficient registration, ...   
2024/07/11 6:57:50 pm GMT+5:30 he event was a delightful experience overall. I...  Suggestions for Improvement:\n\nStreamline the...   

                                                                                   sentiment_score  \
Timestamp                      How was your experience at the event? what all ...                4   
2024/07/11 6:47:37 pm GMT+5:30 The event was enjoyable overall. There were sev...                4   
2024/07/11 6:57:50 pm GMT+5:30 he event was a delightful experience overall. I...                4   

                                              